In [3]:
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### This code will produce arbitrary samples of binary deep eutectic solvents, taking the minimum and maximum mole fractions of each component, number of samples desired to make, and number of trials for the k-means clustering (should be much greater than the number of samples desired to produce). This can easily be modified for ternary deep eutectic solvents. 

In [4]:
def Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array([min_QAS, min_HBD])
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array([max_QAS, max_HBD])

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)



### Generate binary DES with lower mole fractions of 0.2, 0.3 (QAS, HBD) and upper mole fractions of 0.7, 0.8 (QAS, HBD). Produce 96 samples with 192 trials for the clustering.

In [5]:
Binary_DES_Generator(0.2, 0.3, 0.7, 0.8, 96, 192)

 3840 feasible DES samples generated, clustered into 96 samples


array([[0.62904619, 0.37095381],
       [0.38713758, 0.61286242],
       [0.48987807, 0.51012193],
       [0.27893925, 0.72106075],
       [0.54608789, 0.45391211],
       [0.45091607, 0.54908393],
       [0.21388443, 0.78611557],
       [0.34091201, 0.65908799],
       [0.5899394 , 0.4100606 ],
       [0.41683876, 0.58316124],
       [0.51850374, 0.48149626],
       [0.46902422, 0.53097578],
       [0.31393509, 0.68606491],
       [0.67526486, 0.32473514],
       [0.24794668, 0.75205332],
       [0.36646121, 0.63353879],
       [0.57055141, 0.42944859],
       [0.6482374 , 0.3517626 ],
       [0.42963553, 0.57036447],
       [0.40241867, 0.59758133],
       [0.6107284 , 0.3892716 ],
       [0.30340575, 0.69659425],
       [0.50532491, 0.49467509],
       [0.53536481, 0.46463519],
       [0.2623436 , 0.7376564 ],
       [0.35289257, 0.64710743],
       [0.32445484, 0.67554516],
       [0.56095495, 0.43904505],
       [0.37772885, 0.62227115],
       [0.45795393, 0.54204607],
       [0.

### May also save the output as a variable. Note you will get different mole fractions each time so be sure to keep track if you need them.

In [7]:
A = Binary_DES_Generator(0.3, 0.3, 0.7, 0.8, 96, 192)
A 

 3803 feasible DES samples generated, clustered into 96 samples


array([[0.44203553, 0.55796447],
       [0.57893562, 0.42106438],
       [0.38902215, 0.61097785],
       [0.5063012 , 0.4936988 ],
       [0.33612923, 0.66387077],
       [0.62723886, 0.37276114],
       [0.54744113, 0.45255887],
       [0.47238423, 0.52761577],
       [0.67125296, 0.32874704],
       [0.41705557, 0.58294443],
       [0.36297329, 0.63702671],
       [0.59972638, 0.40027362],
       [0.52165906, 0.47834094],
       [0.30206918, 0.69793082],
       [0.48846361, 0.51153639],
       [0.45501731, 0.54498269],
       [0.5593325 , 0.4406675 ],
       [0.40400121, 0.59599879],
       [0.52986776, 0.47013224],
       [0.43390216, 0.56609784],
       [0.66212652, 0.33787348],
       [0.32443714, 0.67556286],
       [0.34493563, 0.65506437],
       [0.63604375, 0.36395625],
       [0.61190174, 0.38809826],
       [0.37682433, 0.62317567],
       [0.44998449, 0.55001551],
       [0.46545547, 0.53454453],
       [0.49880621, 0.50119379],
       [0.5921913 , 0.4078087 ],
       [0.

### The next function will combine the previous function with code to convert mole fractions to a list of lists of volumes that can be directly inputed into code for sample preparation in a pipetting robot (OT-2).

In [8]:
def convert_mole_fractions_to_volumes(stock_QAS, stock_HBD, min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    DES_mole_fractions = Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials)
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 125  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [9]:
convert_mole_fractions_to_volumes(3, 4, 0.3, 0.2, 0.8, 0.7, 96, 192 )

 3840 feasible DES samples generated, clustered into 96 samples


/Users/studentuser/miniconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[[68.47018233346489,
  83.71805032725088,
  100.3692515962536,
  58.05921529489715,
  76.53520262397564,
  91.4801264292444,
  63.930181928401744,
  50.05408853790307,
  72.2409474350069,
  79.55160874356513,
  95.63590960933516,
  87.87511304448343,
  55.1414166020798,
  74.68393787588656,
  86.09077496450911,
  103.12914382957672,
  60.72457748405795,
  82.14500759192671,
  66.42252382899207,
  92.90941448088861,
  70.77680749714382,
  98.12477371116259,
  89.22220822463443,
  65.30267472738184,
  78.50783156676623,
  47.88487148986087,
  54.496117017316564,
  96.22791651147021,
  73.6261511649566,
  80.18346539680904,
  62.58576134201011,
  59.329811270804,
  52.21890087569521,
  101.57617720556009,
  76.10644640713802,
  69.82762309172917,
  94.31246507087623,
  90.36084827652702,
  87.29516105429865,
  84.98621714040593,
  77.89580899571257,
  81.21873943308997,
  99.27650676990835,
  83.20823163807371,
  79.05823897052721,
  57.2870708618696,
  96.83655116150038,
  62.05345752331

### Alternatively, you could use this next function if you already have a list you want to convert (as before when we saved the output of the des mole fractions)

In [10]:
def convert_mole_fractions_to_volumes_2(DES_mole_fractions, stock_QAS, stock_HBD):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 150  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [11]:
convert_mole_fractions_to_volumes_2(A, 3, 2)

[[51.842260173240646,
  71.7373263963853,
  44.69844318255981,
  60.90966586436342,
  37.85422715113997,
  79.3049314688563,
  66.96366857528196,
  56.06677676459793,
  86.47392057243944,
  48.43955213342178,
  41.293471545947426,
  74.95725165968821,
  63.14616109058664,
  33.588983043446476,
  58.346389856751344,
  53.636982373825596,
  68.75158073580948,
  46.68737218380926,
  64.35296260388004,
  50.72707930973579,
  84.96522938162246,
  36.37781937294264,
  38.97483216554685,
  80.71770640417067,
  76.86891523294268,
  43.09558858038532,
  52.93902969901735,
  55.09338671581768,
  59.82817649006162,
  73.78384709739849,
  69.8299090212375,
  49.29584498425155,
  89.52390281843728,
  56.99505362887675,
  83.19799512754442,
  65.6750714451219,
  62.2257392250458,
  45.697230061037736,
  43.70251583510585,
  47.16646867193291,
  40.357687068316366,
  35.04613978552234,
  73.13586873157942,
  77.70184066957407,
  48.014748705930714,
  67.59096787783079,
  59.309272369560524,
  70.4457